Plan:
1. Angelas Code in meinen übersetzen
2. aus Angelas Code alle möglichen Mutationen rausschreiben, aber nur einmal
2. --> Liste mit allen vorkommenden Mutationen
3. Funktion schreiben, die für jede dieser (einzeln!) alle Sequenzen raussucht in der diese vorkommt
4. --> ein dataframe, mit mutation als spaltenname und allen sequenzen wo es drin vorkommt als Reihen drunter und noch einer Spalte mit mutcount dahinter, pro Mutation werden zwei Spalten an den dataframe angehängt
5. (dadurch können Reihen spezifisch nach mutcount angeprochen werden)
6. innerhalb einer Mutationsspalte, alle mit dem selben mutcount: fscore vergleich : Varianz der Werte als Maß für impact
-> je unwichtiger die Mutation, desto weniger macht sie was beim fscore, desto weniger sollten sich die Werte unterscheiden, desto kleiner sollte die Varianz sein

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)

In [3]:
# Datensatz einlesen
original_dms_data = pd.read_csv('/Users/liza/Desktop/Bioinfo Project/DMS_data/GFP_AEQVI_Sarkisyan_2016.csv')
# split first column of df into multiple columns
original_dms_data_col = original_dms_data
only_mutants = original_dms_data["mutant"].to_frame()
original_dms_data_col[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15']] = original_dms_data_col['mutant'].str.split(':', 15, expand=True)
original_dms_data_col.head()


/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_29859/404050330.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  original_dms_data_col[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15']] = original_dms_data_col['mutant'].str.split(':', 15, expand=True)


,mutant,mutated_sequence,DMS_score,DMS_score_bin,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15
0,K3R:V55A:Q94R:A110T:D117G:M153K:D216A,MSRGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,1.301030,0,K3R,V55A,Q94R,A110T,D117G,M153K,D216A,None,None,None,None,None,None,None,None
1,K3Q:V16A:I167T:L195Q,MSQGEELFTGVVPILAELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.137350,1,K3Q,V16A,I167T,L195Q,None,None,None,None,None,None,None,None,None,None,None
2,K3Q:Y143C:N164D:S205P:A227T,MSQGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,1.553913,0,K3Q,Y143C,N164D,S205P,A227T,None,None,None,None,None,None,None,None,None,None
3,K3Q:Y143N:V193A,MSQGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.404237,1,K3Q,Y143N,V193A,None,None,None,None,None,None,None,None,None,None,None,None
4,K3R,MSRGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.738586,1,K3R,None,None,None,None,None,None,None,None,None,None,None,None,None,None


Dataframe erstellen aus den Spalten: (WORKING_DATAFRAME)
- Mutationsanzahl aus df_mutation_counts
- fscore aus original_dms_data_col
- m1 bis b11 aus original_dms_data_col

In [4]:
# zählen wieviele Mutationen jede Sequenz hat
list_mut_count_in_progress = []
for i in range(len(original_dms_data['mutant'])):
    list_mut_count_in_progress.append(original_dms_data['mutant'].iloc[i].count(':'))
list_mut_count_prae = np.array(list_mut_count_in_progress)
list_mut_count = (list_mut_count_prae + 1)
df_mutation_counts = pd.DataFrame(list_mut_count)

In [5]:
#zusammenfügen
working_dataframe_prae = pd.concat([original_dms_data_col, df_mutation_counts], axis="columns")
#alle Spalten löschen die stören
working_dataframe = working_dataframe_prae.drop(['mutant', 'mutated_sequence', 'DMS_score_bin'], axis=1)
working_dataframe.rename(columns={working_dataframe.columns[16]: 'mut_count'}, inplace=True)
working_dataframe.head()

,DMS_score,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15,mut_count
0,1.301030,K3R,V55A,Q94R,A110T,D117G,M153K,D216A,None,None,None,None,None,None,None,None,7
1,3.137350,K3Q,V16A,I167T,L195Q,None,None,None,None,None,None,None,None,None,None,None,4
2,1.553913,K3Q,Y143C,N164D,S205P,A227T,None,None,None,None,None,None,None,None,None,None,5
3,3.404237,K3Q,Y143N,V193A,None,None,None,None,None,None,None,None,None,None,None,None,3
4,3.738586,K3R,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1


In [6]:
#alle möglichen Mutationen rausschreiben in den Spalten
from pandas import unique
working_dataframe_only_ms = working_dataframe.loc[:, ["m1", "m2", "m3", 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15']]
working_dataframe_only_ms.head()


,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15
0,K3R,V55A,Q94R,A110T,D117G,M153K,D216A,None,None,None,None,None,None,None,None
1,K3Q,V16A,I167T,L195Q,None,None,None,None,None,None,None,None,None,None,None
2,K3Q,Y143C,N164D,S205P,A227T,None,None,None,None,None,None,None,None,None,None
3,K3Q,Y143N,V193A,None,None,None,None,None,None,None,None,None,None,None,None
4,K3R,None,None,None,None,None,None,None,None,None,None,None,None,None,None


1. alle Werte des DataFrames mit values.flatten() in eine eindimensionale Liste umgewandeln
2. tolist() verwendet, um die Liste in eine normale Python-Liste umzuwandeln
3.  Liste in ein Set umgewandeln (nur eindeutige Elemente)
4. set zurück in Liste

In [7]:
all_possible_mutations = working_dataframe_only_ms.values.flatten().tolist()
all_possible_mutations = list(set(all_possible_mutations))

In [8]:
only_mutants_list = only_mutants['mutant']
only_mutants_list

0        K3R:V55A:Q94R:A110T:D117G:M153K:D216A
1                         K3Q:V16A:I167T:L195Q
2                  K3Q:Y143C:N164D:S205P:A227T
3                              K3Q:Y143N:V193A
4                                          K3R
                         ...                  
51709                                    Y237N
51710                                    Y237C
51711                                    Y237D
51712                                    Y237F
51713                                    Y237H
Name: mutant, Length: 51714, dtype: object

In [16]:
def checking_existence(soll_gecheckt_werden, Datenset):
    list_of_dfs = []

    for i in soll_gecheckt_werden:
        new_column_name = f'{i}'
        new_column_values = [ i in x for x in Datenset]
        new_df = pd.DataFrame({new_column_name: new_column_values})
        list_of_dfs.append(new_df)

    result_how_often = pd.concat(list_of_dfs, axis=1)
    print(result_how_often)

checking_existence(all_possible_mutations, only_mutants_list)


TypeError: 'in <string>' requires string as left operand, not NoneType

In [16]:
list_of_dfs = []

# For-Loop zum Erstellen und Hinzufügen eines neuen Columns in jedem Durchlauf
for i in all_possible_mutations:
    new_column_name = f'{i}'
    new_column_values = [ i in x for x in only_mutants_list]
    new_df = pd.DataFrame({new_column_name: new_column_values})
    list_of_dfs.append(new_df)

# Verketten der DataFrames in der Liste
result_how_often = pd.concat(list_of_dfs, axis=1)

print(result_how_often)

TypeError: 'in <string>' requires string as left operand, not NoneType

In [10]:
import pandas as pd

column_values = only_mutants['mutant'].tolist()
list_of_dfs = [pd.DataFrame({str(i): [i in x for x in column_values]}) for i in all_possible_mutations]
result_how_often = pd.concat(list_of_dfs, axis=1)

print(result_how_often)

TypeError: 'in <string>' requires string as left operand, not NoneType

In [10]:

list_of_dfs = []

# For-Loop zum Erstellen und Hinzufügen eines neuen Columns in jedem Durchlauf
for i in all_possible_mutations:
    new_column_name = f'{i}'
    new_column_values = [ only_mutants_list.str.contains('i', regex=False)]
    new_df = pd.DataFrame({new_column_name: new_column_values})
    list_of_dfs.append(new_df)

# Verketten der DataFrames in der Liste
result_how_often = pd.concat(list_of_dfs, axis=1)

result_how_often

,M153S,N144A,L236P,H199Y,V55G,G51D,G116D,K140L,N135C,I14S,E17Q,G67A,A110D,R73C,N212Y,E6D,K45V,F46T,K166I,V16F,E115D,I98F,W57S,S208R,D21H,...,Q183P,I188M,D103N,I128S,G134V,H169C,M78E,K166V,Q80E,F8Y,T59P,T203A,A110V,E222Q,G31D,K166Q,H217D,K101W,V22E,E222R,H217Y,F130I,I123D,Y200S,D155N
0,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...


In [19]:
s = result_how_often['E95A'][0]
if s.any():
    print("Es gibt True-Werte in der Serie.")
else:
    print("Es gibt keine True-Werte in der Serie.")

Es gibt keine True-Werte in der Serie.


was funktioniert: er macht für jede Mutation eine Spalte und benennt sie nach der Mutation
und hängt sie alle aneinander als dataframe
5,6,8-14

was nicht funktioniert: die erkennung des substrings -> er gibt immer false aus

In [15]:
list_of_dfs = []

# For-Loop zum Erstellen und Hinzufügen eines neuen Columns in jedem Durchlauf
for i in all_possible_mutations:
    new_column_name = f'{i}'
    new_column_values = [only_mutants_list.iloc[range(len(only_mutants_list))].str.contains('i')]
    new_df = pd.DataFrame({new_column_name: new_column_values})
    list_of_dfs.append(new_df)

# Verketten der DataFrames in der Liste
result_how_often = pd.concat(list_of_dfs, axis=1)

result_how_often

,D76A,E95A,R96H,K162R,L60F,H148F,I171V,E5G,G228W,R96P,L60I,I161A,K166R,G40V,I167D,K166I,N23D,D76G,H181L,T118K,L236Q,K158R,I123T,T97P,S208T,...,F71D,P56A,N146V,T50P,D82A,Y74S,T9N,K166Q,A179G,Y66D,G20C,H139N,S175R,I128Y,N159V,I188T,I161T,K45T,L201H,D197S,Y92C,D234V,R168C,I14N,G10E
0,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...,0 False 1 False 2 False 3...


In [20]:
# code für einzelne Mutation derfunktioniert wie er soll
resultingg_mutss = pd.DataFrame()
substring = 'V193A'

# Boolianischen Wert ausgeben, ob der Substring in einer Zeile existiert
resultingg_mutss['V193A'] = only_mutants['mutant'].apply(lambda x: substring in x)

print(resultingg_mutss)
# gibt aber die ganze zeile aus nicht den index ---> mit df.loc !! dann die zeilennummern

       V193A
0      False
1      False
2      False
3       True
4      False
...      ...
51709  False
51710  False
51711  False
51712  False
51713  False

[51714 rows x 1 columns]
